<a href="https://colab.research.google.com/github/Amna-ismail/Chatbot-Deep-Learning-/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import json
import random


In [ ]:
data = {
    "intents": [
        {"tag": "greeting", "patterns": ["Hi", "Hello", "How are you", "Is anyone there?", "Good day"], "responses": ["Hello!", "Hi there!", "Greetings!", "How can I help you?"]},
        {"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye"], "responses": ["Goodbye!", "See you later!", "Have a great day!"]},
        {"tag": "thanks", "patterns": ["Thanks", "Thank you", "That's helpful"], "responses": ["You're welcome!", "Happy to help!", "Anytime!"]},
        {"tag": "noanswer", "patterns": [], "responses": ["Sorry, I don't understand."]},
    ]
}

# Save data to a JSON file for easy loading
with open('chatbot_data.json', 'w') as outfile:
    json.dump(data, outfile)


In [ ]:
# Load the JSON file
with open('chatbot_data.json') as file:
    data = json.load(file)

# Initialize lists for patterns and responses
patterns = []
responses = []
classes = []
documents = []

# Extract data
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        responses.append(intent['responses'])
        documents.append((pattern, intent['tag']))

    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Tokenize the patterns
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index

# Convert patterns to sequences
sequences = tokenizer.texts_to_sequences(patterns)
max_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Create target variables
labels = [classes.index(doc[1]) for doc in documents]
labels = np.array(labels)

# One-hot encode labels
labels = tf.keras.utils.to_categorical(labels, num_classes=len(classes))


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=2000, output_dim=128, input_length=max_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(classes), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 128)            256000    
                                                                 
 lstm (LSTM)                 (None, 3, 128)            131584    
                                                                 
 dropout (Dropout)           (None, 3, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 4)                 5

In [ ]:
history = model.fit(padded_sequences, labels, epochs=10, batch_size=5, verbose=1)


Epoch 1/10
3/3 [==============================] - 4s 20ms/step - loss: 1.3848 - accuracy: 0.4545
Epoch 2/10
3/3 [==============================] - 0s 15ms/step - loss: 1.3739 - accuracy: 0.4545
Epoch 3/10
3/3 [==============================] - 0s 15ms/step - loss: 1.3640 - accuracy: 0.5455
Epoch 4/10
3/3 [==============================] - 0s 15ms/step - loss: 1.3535 - accuracy: 0.4545
Epoch 5/10
3/3 [==============================] - 0s 16ms/step - loss: 1.3417 - accuracy: 0.4545
Epoch 6/10
3/3 [==============================] - 0s 16ms/step - loss: 1.3242 - accuracy: 0.4545
Epoch 7/10
3/3 [==============================] - 0s 15ms/step - loss: 1.2968 - accuracy: 0.4545
Epoch 8/10
3/3 [==============================] - 0s 14ms/step - loss: 1.2788 - accuracy: 0.4545
Epoch 9/10
3/3 [==============================] - 0s 14ms/step - loss: 1.2451 - accuracy: 0.4545
Epoch 10/10
3/3 [==============================] - 0s 14ms/step - loss: 1.2228 - accuracy: 0.4545


In [ ]:
model.save('chatbot_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load the model
model = tf.keras.models.load_model('chatbot_model.h5')

# Define a function to clean up user input
def clean_up_sentence(sentence):
    sentence_words = tokenizer.texts_to_sequences([sentence])
    sentence_words = pad_sequences(sentence_words, maxlen=max_length, padding='post')
    return sentence_words

# Define a function to classify user input
def classify(sentence):
    sentence_words = clean_up_sentence(sentence)
    prediction = model.predict(sentence_words)[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, res] for i, res in enumerate(prediction) if res > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Define a function to get a response
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Define the chatbot response function
def chatbot_response(text):
    intents = classify(text)
    response = get_response(intents, data)
    return response


In [ ]:
print("Start talking with the bot (type 'quit' to stop)!")
while True:
    message = input("")
    if message.lower() == "quit":
        break
    response = chatbot_response(message)
    print(message)
    print(response)


Start talking with the bot (type 'quit' to stop)!
